In [2]:
import pandas as pd
from pandas.tseries.offsets import BDay
import statsmodels.api as sm
import numpy as np

### 1. Merge COT Data with Futures Return

#### WTI Crude Oil

In [ ]:
# --- Load and clean the COT data ---
df_cot = pd.read_csv("WTI.csv")
df_cot["COT_Date"] = pd.to_datetime(df_cot["Report_Date_as_YYYY_MM_DD"])

# --- Load and clean front-month futures prices (CL1) ---
df_price_raw = pd.read_excel("CL1.xlsx", skiprows=6)
df_price_clean = df_price_raw[["Date", "PX_SETTLE"]].dropna()
df_price_clean.columns = ["Date", "Settle"]
df_price_clean["Date"] = pd.to_datetime(df_price_clean["Date"])

# --- Build expiry calendar for WTI contracts ---
# Note: CME WTI contracts expire 3 business days before the 25th of the month prior to delivery.
def get_expiry_dates(start, end):
    dates = pd.date_range(start, end, freq="MS")
    expiry = [pd.Timestamp(y, m, 25) - BDay(3) for y, m in zip(dates.year, dates.month)]
    expiry_df = pd.DataFrame({"Expiry": expiry})
    expiry_df["YearMonth"] = expiry_df["Expiry"].dt.to_period("M")
    return expiry_df
expiry_calendar = get_expiry_dates(df_price_clean["Date"].min(), df_price_clean["Date"].max())

# --- Assign each price to a contract month and flag rollover weeks ---
df_price_clean["YearMonth"] = df_price_clean["Date"].dt.to_period("M")
df_price_clean = df_price_clean.merge(expiry_calendar, on="YearMonth", how="left")
# Compute days-to-expiry
df_price_clean["DaysToExpiry"] = (df_price_clean["Expiry"] - df_price_clean["Date"]).dt.days
df_price_clean["RolloverRisk"] = df_price_clean["DaysToExpiry"].between(0, 5) # Any price date within 5 calendar days of expiry is flagged

# --- Merge COT data with prices and drop rollover weeks ---
df_merged = pd.merge(df_cot, df_price_clean, left_on="COT_Date", right_on="Date", how="inner")
df_merged_clean = df_merged[~df_merged["RolloverRisk"]].copy()

# --- Calculate weekly returns ---
df_merged_clean = df_merged_clean.sort_values("COT_Date")
df_merged_clean["Settle_t+1"] = df_merged_clean["Settle"].shift(-1)
df_merged_clean["Return"] = (df_merged_clean["Settle_t+1"] - df_merged_clean["Settle"]) / df_merged_clean["Settle"]



/var/folders/q3/lw2ccwxs3gbd91grj537_4y40000gn/T/ipykernel_18229/113116674.py:3: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_cot["COT_Date"] = pd.to_datetime(df_cot["Report_Date_as_YYYY_MM_DD"])


In [ ]:
useful_columns = [
    "COT_Date", "Settle", "Settle_t+1", "Return","Open_Interest_All",
    # Producer/Merchant
    "Prod_Merc_Positions_Long_All", "Prod_Merc_Positions_Short_All", "Prod_Net",
    # Swap Dealer
    "Swap_Positions_Long_All", "Swap__Positions_Short_All", "Swap_Net",
    # Managed Money
    "M_Money_Positions_Long_All", "M_Money_Positions_Short_All", "MM_Net",
    # Other Reportables
    "Other_Rept_Positions_Long_All", "Other_Rept_Positions_Short_All","Other_Net",
    # Non-Reportables
    "NonRept_Positions_Long_All", "NonRept_Positions_Short_All","NonRep_Net"
]
df_merged_clean["MM_Net"] = df_merged_clean["M_Money_Positions_Long_All"] - df_merged_clean["M_Money_Positions_Short_All"]
df_merged_clean["Swap_Net"] = df_merged_clean["Swap_Positions_Long_All"] - df_merged_clean["Swap__Positions_Short_All"]
df_merged_clean["Prod_Net"] = df_merged_clean["Prod_Merc_Positions_Long_All"] - df_merged_clean["Prod_Merc_Positions_Short_All"]
df_merged_clean["Other_Net"] = df_merged_clean["Other_Rept_Positions_Long_All"] - df_merged_clean["Other_Rept_Positions_Short_All"]
df_merged_clean["NonRep_Net"] = df_merged_clean["NonRept_Positions_Long_All"] - df_merged_clean["NonRept_Positions_Short_All"]

df_WTI_final = df_merged_clean[useful_columns].copy()

#### RBOB Gasoline

In [ ]:
# --- Load and prepare RBOB position data ---
df_gas = pd.read_csv("Gasoline.csv")
df_gas["COT_Date"] = pd.to_datetime(df_gas["Report_Date_as_YYYY_MM_DD"])

# --- Load and prepare XB1 front-month futures price data ---
df_xb1_raw = pd.read_excel("XB1.xlsx", skiprows=6)
df_xb1_clean = df_xb1_raw[["Date", "PX_SETTLE"]].dropna()
df_xb1_clean.columns = ["Date", "Settle"]
df_xb1_clean["Date"] = pd.to_datetime(df_xb1_clean["Date"])

# --- Build expiry calendar for RBOB Gasoline ---
# RBOB contracts expire on the last business day of the month before the contract month
def get_rbob_expiry_dates(start, end):
    dates = pd.date_range(start, end, freq="MS")
    expiry = [pd.Timestamp(y, m, 1) - BDay(1) for y, m in zip(dates.year, dates.month)]
    expiry_df = pd.DataFrame({"Expiry": expiry})
    expiry_df["YearMonth"] = expiry_df["Expiry"].dt.to_period("M")
    return expiry_df

rbob_expiry_calendar = get_rbob_expiry_dates(df_xb1_clean["Date"].min(), df_xb1_clean["Date"].max())

# --- Flag rollover risk ---
df_xb1_clean["YearMonth"] = df_xb1_clean["Date"].dt.to_period("M")
df_xb1_clean = df_xb1_clean.merge(rbob_expiry_calendar, on="YearMonth", how="left")
df_xb1_clean["DaysToExpiry"] = (df_xb1_clean["Expiry"] - df_xb1_clean["Date"]).dt.days
df_xb1_clean["RolloverRisk"] = df_xb1_clean["DaysToExpiry"].between(0, 5)

# --- Merge COT data with XB1 price data and exclude rollover weeks ---
df_merged = pd.merge(df_gas, df_xb1_clean, left_on="COT_Date", right_on="Date", how="inner")
df_merged_clean = df_merged[~df_merged["RolloverRisk"]].copy()

# --- Calculate weekly returns ---
df_merged_clean = df_merged_clean.sort_values("COT_Date")
df_merged_clean["Settle_t+1"] = df_merged_clean["Settle"].shift(-1)
df_merged_clean["Return"] = (df_merged_clean["Settle_t+1"] - df_merged_clean["Settle"]) / df_merged_clean["Settle"]

# --- Calculate net positions for all five trader types ---
df_merged_clean["MM_Net"] = df_merged_clean["M_Money_Positions_Long_All"] - df_merged_clean["M_Money_Positions_Short_All"]
df_merged_clean["Swap_Net"] = df_merged_clean["Swap_Positions_Long_All"] - df_merged_clean["Swap__Positions_Short_All"]
df_merged_clean["Prod_Net"] = df_merged_clean["Prod_Merc_Positions_Long_All"] - df_merged_clean["Prod_Merc_Positions_Short_All"]
df_merged_clean["Other_Net"] = df_merged_clean["Other_Rept_Positions_Long_All"] - df_merged_clean["Other_Rept_Positions_Short_All"]
df_merged_clean["NonRep_Net"] = df_merged_clean["NonRept_Positions_Long_All"] - df_merged_clean["NonRept_Positions_Short_All"]

useful_columns = [
    "COT_Date", "Settle", "Settle_t+1", "Return","Open_Interest_All",
    "Prod_Merc_Positions_Long_All", "Prod_Merc_Positions_Short_All", "Prod_Net",
    "Swap_Positions_Long_All", "Swap__Positions_Short_All", "Swap_Net",
    "M_Money_Positions_Long_All", "M_Money_Positions_Short_All", "MM_Net",
    "Other_Rept_Positions_Long_All", "Other_Rept_Positions_Short_All", "Other_Net",
    "NonRept_Positions_Long_All", "NonRept_Positions_Short_All", "NonRep_Net"
]

df_XB_final = df_merged_clean[useful_columns].copy()
df_XB_final.to_excel("XB_merged.xlsx", index=False)

/var/folders/q3/lw2ccwxs3gbd91grj537_4y40000gn/T/ipykernel_18229/3021542581.py:3: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_gas["COT_Date"] = pd.to_datetime(df_gas["Report_Date_as_YYYY_MM_DD"])


### 2. Feature transforms (net positions as % change and z-scores)

In [13]:
# --- Convert to % change ---
net_cols = ["MM_Net", "Swap_Net", "Prod_Net", "Other_Net", "NonRep_Net"]

# --- For RBOB ---
for col in net_cols:
    df_XB_final[f"{col}_Chg"] = df_XB_final[col].pct_change()
    df_XB_final[f"{col}_Chg"] =df_XB_final[f"{col}_Chg"].replace([float("inf"), float("-inf")], pd.NA)

# --- For WTI ---
for col in net_cols:
    df_WTI_final[f"{col}_Chg"] = df_WTI_final[col].pct_change()
    df_WTI_final[f"{col}_Chg"] = df_WTI_final[f"{col}_Chg"].replace([float("inf"), float("-inf")], pd.NA)

df_WTI_final.to_excel("WTI_merged.xlsx", index=False)
df_XB_final.to_excel("XB_merged.xlsx", index=False)

In [14]:
# --- Convert to z-scores ---

# Rolling Z‑scores (52‑week window)
window = 52
def add_z(df):
    for col in net_cols:
        df[col+"_z"] = (df[col] - df[col].rolling(window, min_periods=26).mean()) / df[col].rolling(window, min_periods=26).std()
    return df

df_WTI = add_z(df_WTI_final)
df_XB  = add_z(df_XB_final)

z_cols = [c+"_z" for c in net_cols]

### 3. Regression Model

In [ ]:
def get_R2_and_models(df, label):
    R2s = {}
    models = {}
    for h in [1,2,3]:
        tgt = f"Ret_t{h}"
        df[tgt] = df["Return"].shift(-h)
        reg = df[[tgt]+z_cols].dropna()
        X = sm.add_constant(reg[z_cols])
        res = sm.OLS(reg[tgt], X).fit()
        R2s[h] = res.rsquared
        models[h] = res
    return R2s, models

R2_wti, models_wti = get_R2_and_models(df_WTI,"WTI")
R2_xb , models_xb  = get_R2_and_models(df_XB ,"RBOB")

R2_table_WTI = pd.DataFrame({
    "Lag": ["t+1","t+2","t+3"],
    "R2": [round(R2_wti[1],4), round(R2_wti[2],4), round(R2_wti[3],4)]
})
R2_table_XB = pd.DataFrame({
    "Lag": ["t+1","t+2","t+3"],
    "R2": [round(R2_xb[1],4), round(R2_xb[2],4), round(R2_xb[3],4)]
})

# Select best horizon
best_wti_h = max(R2_wti, key=R2_wti.get)
best_xb_h  = max(R2_xb , key=R2_xb.get)

def beta_table(res, label, horizon):
    rows = []
    for var in z_cols:
        rows.append({
            "Predictor": var.replace("_Net_z","").replace("_z",""),
            "Beta": round(res.params[var],4),
            "p_value": round(res.pvalues[var],4)
        })
    return pd.DataFrame(rows)

beta_wti = beta_table(models_wti[best_wti_h], "WTI", best_wti_h)
beta_xb  = beta_table(models_xb[best_xb_h], "RBOB", best_xb_h)

In [ ]:
R2_table_WTI

,Lag,R2
0,t+1,0.0072
1,t+2,0.0093
2,t+3,0.0058


In [ ]:
R2_table_XB

,Lag,R2
0,t+1,0.0125
1,t+2,0.0140
2,t+3,0.0106


In [ ]:
beta_wti

,Predictor,Beta,p_value
0,MM,0.0043,0.1457
1,Swap,0.0054,0.1822
2,Prod,0.0042,0.3336
3,Other,0.0006,0.8747
4,NonRep,0.0025,0.3043


In [ ]:
beta_xb

,Predictor,Beta,p_value
0,MM,-0.0054,0.3298
1,Swap,0.0004,0.8901
2,Prod,-0.0108,0.0478
3,Other,-0.0013,0.7151
4,NonRep,-0.0033,0.3003


### 4. Release Date Regression

In [ ]:
from pandas.tseries.holiday import USFederalHolidayCalendar
from pandas.tseries.offsets import CustomBusinessDay
us_bd = CustomBusinessDay(calendar=USFederalHolidayCalendar())


In [ ]:
df_cot_release = df_cot.copy()
df_cot_release["Release_Date"] = df_cot_release["COT_Date"] + 3 * us_bd

df_merged_release = pd.merge(
    df_cot_release,
    df_price_clean,
    left_on="Release_Date",
    right_on="Date",
    how="inner"
)

df_merged_clean_release = df_merged_release[~df_merged_release["RolloverRisk"]].copy()

df_merged_clean_release = df_merged_clean_release.sort_values("Release_Date")
df_merged_clean_release["Settle_t+1"] = df_merged_clean_release["Settle"].shift(-1)
df_merged_clean_release["Return"] = (
    df_merged_clean_release["Settle_t+1"] - df_merged_clean_release["Settle"]
) / df_merged_clean_release["Settle"]

df_merged_clean_release["MM_Net"] = df_merged_clean_release["M_Money_Positions_Long_All"] - df_merged_clean_release["M_Money_Positions_Short_All"]
df_merged_clean_release["Swap_Net"] = df_merged_clean_release["Swap_Positions_Long_All"] - df_merged_clean_release["Swap__Positions_Short_All"]
df_merged_clean_release["Prod_Net"] = df_merged_clean_release["Prod_Merc_Positions_Long_All"] - df_merged_clean_release["Prod_Merc_Positions_Short_All"]
df_merged_clean_release["Other_Net"] = df_merged_clean_release["Other_Rept_Positions_Long_All"] - df_merged_clean_release["Other_Rept_Positions_Short_All"]
df_merged_clean_release["NonRep_Net"] = df_merged_clean_release["NonRept_Positions_Long_All"] - df_merged_clean_release["NonRept_Positions_Short_All"]

df_WTI_release_final = df_merged_clean_release[useful_columns].copy()
net_cols = ["MM_Net", "Swap_Net", "Prod_Net", "Other_Net", "NonRep_Net"]
df_WTI_release = add_z(df_WTI_release_final)
R2_wti_release, models_wti_release = get_R2_and_models(df_WTI_release, "WTI_Release")

R2_table_WTI_release = pd.DataFrame({
    "Lag": ["t+1", "t+2", "t+3"],
    "R2": [round(R2_wti_release[1],4), round(R2_wti_release[2],4), round(R2_wti_release[3],4)]
})
R2_table_WTI_release

/var/folders/q3/lw2ccwxs3gbd91grj537_4y40000gn/T/ipykernel_18229/1767991986.py:2: PerformanceWarning: Non-vectorized DateOffset being applied to Series or DatetimeIndex.
  df_cot_release["Release_Date"] = df_cot_release["COT_Date"] + 3 * us_bd


,Lag,R2
0,t+1,0.0058
1,t+2,0.0103
2,t+3,0.0086


In [ ]:
df_gas_release = df_gas.copy()
df_gas_release["Release_Date"] = df_gas_release["COT_Date"] + 3 * us_bd

df_merged_xb_release = pd.merge(
    df_gas_release,
    df_xb1_clean,
    left_on="Release_Date",
    right_on="Date",
    how="inner"
)

df_merged_xb_clean_release = df_merged_xb_release[~df_merged_xb_release["RolloverRisk"]].copy()

df_merged_xb_clean_release = df_merged_xb_clean_release.sort_values("Release_Date")
df_merged_xb_clean_release["Settle_t+1"] = df_merged_xb_clean_release["Settle"].shift(-1)
df_merged_xb_clean_release["Return"] = (
    df_merged_xb_clean_release["Settle_t+1"] - df_merged_xb_clean_release["Settle"]
) / df_merged_xb_clean_release["Settle"]

df_merged_xb_clean_release["MM_Net"] = df_merged_xb_clean_release["M_Money_Positions_Long_All"] - df_merged_xb_clean_release["M_Money_Positions_Short_All"]
df_merged_xb_clean_release["Swap_Net"] = df_merged_xb_clean_release["Swap_Positions_Long_All"] - df_merged_xb_clean_release["Swap__Positions_Short_All"]
df_merged_xb_clean_release["Prod_Net"] = df_merged_xb_clean_release["Prod_Merc_Positions_Long_All"] - df_merged_xb_clean_release["Prod_Merc_Positions_Short_All"]
df_merged_xb_clean_release["Other_Net"] = df_merged_xb_clean_release["Other_Rept_Positions_Long_All"] - df_merged_xb_clean_release["Other_Rept_Positions_Short_All"]
df_merged_xb_clean_release["NonRep_Net"] = df_merged_xb_clean_release["NonRept_Positions_Long_All"] - df_merged_xb_clean_release["NonRept_Positions_Short_All"]

df_XB_release_final = df_merged_xb_clean_release[useful_columns].copy()

df_XB_release = add_z(df_XB_release_final)

R2_xb_release, models_xb_release = get_R2_and_models(df_XB_release, "RBOB_Release")
R2_table_XB_release = pd.DataFrame({
    "Lag": ["t+1", "t+2", "t+3"],
    "R2": [round(R2_xb_release[1], 4), round(R2_xb_release[2], 4), round(R2_xb_release[3], 4)]
})
R2_table_XB_release



/var/folders/q3/lw2ccwxs3gbd91grj537_4y40000gn/T/ipykernel_18229/1088181742.py:2: PerformanceWarning: Non-vectorized DateOffset being applied to Series or DatetimeIndex.
  df_gas_release["Release_Date"] = df_gas_release["COT_Date"] + 3 * us_bd


,Lag,R2
0,t+1,0.0092
1,t+2,0.0123
2,t+3,0.0096


In [ ]:
best_wti_release_h = max(R2_wti_release, key=R2_wti_release.get)
beta_wti_release = beta_table(models_wti_release[best_wti_release_h], "WTI_Release", best_wti_release_h)
beta_wti_release


,Predictor,Beta,p_value
0,MM,0.0045,0.0611
1,Swap,0.0031,0.3527
2,Prod,0.0014,0.6965
3,Other,-0.0006,0.8412
4,NonRep,0.0008,0.7103


In [ ]:
best_xb_release_h = max(R2_xb_release, key=R2_xb_release.get)
beta_xb_release = beta_table(models_xb_release[best_xb_release_h], "RBOB_Release", best_xb_release_h)
beta_xb_release


,Predictor,Beta,p_value
0,MM,-0.0013,0.7974
1,Swap,0.0010,0.6804
2,Prod,-0.0067,0.1716
3,Other,0.0004,0.8887
4,NonRep,-0.0017,0.5525


### 5. Inventory

In [3]:
df_XB  = pd.read_excel("XB_merged.xlsx")
df_WTI = pd.read_excel("WTI_merged.xlsx")

In [6]:
inv = pd.read_csv("wti_inv.csv", index_col=0)
s = inv.iloc[0]
s.index = pd.to_datetime(s.index, format="%b-%y").to_period("M")
s.name = "Inventory"
inv_wti = s.reset_index().rename(columns={"index":"COT_Date"})
inv_wti["COT_Date"] = inv_wti["COT_Date"].dt.strftime("%Y-%m")

inv_wti

,COT_Date,Inventory
0,2009-07,327.2
1,2009-08,317.5
2,2009-09,317.1
3,2009-10,314.4
4,2009-11,318.8
...,...,...
205,2026-08,414.2
206,2026-09,412.7
207,2026-10,423.4
208,2026-11,422.0


In [9]:
inv_xb = pd.read_csv("xb_inv.csv", index_col=0)
g = inv_xb.iloc[0]
g.index = pd.to_datetime(g.index, format="%b-%y").to_period("M")
g.name = "Inventory"
inv_xb = g.reset_index().rename(columns={"index":"COT_Date"})
inv_xb["COT_Date"] = inv_xb["COT_Date"].dt.strftime("%Y-%m")

inv_xb

,COT_Date,Inventory
0,2006-06,213.3
1,2006-07,208.9
2,2006-08,209.0
3,2006-09,214.1
4,2006-10,204.6
...,...,...
242,2026-08,209.0
243,2026-09,206.8
244,2026-10,202.7
245,2026-11,212.1


In [10]:
df_WTI = pd.read_excel("WTI_merged.xlsx", parse_dates=["COT_Date"])
df_WTI = (
    df_WTI
      .set_index("COT_Date")
      .sort_index()
)

delta_cols = [
    "MM_Net_Chg", "Swap_Net_Chg", "Prod_Net_Chg",
    "Other_Net_Chg", "NonRep_Net_Chg"
]

# Monthly return: compound (1+weekly_return) across all weeks in month, then subtract 1
monthly_ret_WTI = (
    df_WTI["Return"].add(1)
      .resample("M")     # month-end; you can also use "ME" for explicit MonthEnd
      .prod()
      .sub(1)
      .rename("Ret_m")
)

# Monthly net position: sum the weekly Δ across the month
monthly_deltas = (
    df_WTI[delta_cols]
      .resample("M")
      .sum()
      # rename columns to shorter Δ names if you like:
      .rename(columns={
          "MM_Net_Chg":"MM_Δ",
          "Swap_Net_Chg":"Swap_Δ",
          "Prod_Net_Chg":"Prod_Δ",
          "Other_Net_Chg":"Other_Δ",
          "NonRep_Net_Chg":"NonRep_Δ"
      })
)

# Combine into a single monthly DataFrame
df_m_wti = pd.concat([monthly_ret_WTI, monthly_deltas], axis=1).dropna()
df_m_wti.index = df_m_wti.index.strftime("%Y-%m")
df_m_wti

<ipython-input-10-b2d32325a39c>:16: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  .resample("M")     # month-end; you can also use "ME" for explicit MonthEnd
<ipython-input-10-b2d32325a39c>:25: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  .resample("M")


,Ret_m,MM_Δ,Swap_Δ,Prod_Δ,Other_Δ,NonRep_Δ
COT_Date,,,,,,
2009-07,0.062323,0.000000,0.000000,0.000000,0.000000,0.000000
2009-08,-0.047186,0.393580,-0.509397,-0.275579,-0.093420,-2.761754
2009-09,0.041587,-0.274340,0.059355,-0.379426,0.364418,8.965208
2009-10,0.123025,0.182130,0.369218,0.783571,-0.622787,-12.040907
2009-11,-0.015452,-0.645380,0.100711,-0.189565,0.438153,1.906155
...,...,...,...,...,...,...
2024-12,0.061624,-0.198787,-0.094822,-0.856296,0.741467,1.772403
2025-01,-0.020875,-0.653224,-0.260872,1.166076,-0.846702,1.938516
2025-02,-0.061073,0.711218,0.479311,1.266449,-0.601381,1.326063


In [11]:
df_XB = pd.read_excel("XB_merged.xlsx", parse_dates=["COT_Date"])
df_XB = (
    df_XB
      .set_index("COT_Date")
      .sort_index()
)

delta_cols = [
    "MM_Net_Chg", "Swap_Net_Chg", "Prod_Net_Chg",
    "Other_Net_Chg", "NonRep_Net_Chg"
]

# Monthly return: compound (1+weekly_return) across all weeks in month, then subtract 1
monthly_ret_XB = (
    df_XB["Return"].add(1)
      .resample("M")     # month-end; you can also use "ME" for explicit MonthEnd
      .prod()
      .sub(1)
      .rename("Ret_m")
)

# Monthly net position: sum the weekly Δ across the month
monthly_deltas = (
    df_XB[delta_cols]
      .resample("M")
      .sum()
      # rename columns to shorter Δ names if you like:
      .rename(columns={
          "MM_Net_Chg":"MM_Δ",
          "Swap_Net_Chg":"Swap_Δ",
          "Prod_Net_Chg":"Prod_Δ",
          "Other_Net_Chg":"Other_Δ",
          "NonRep_Net_Chg":"NonRep_Δ"
      })
)

# Combine into a single monthly DataFrame
df_m_XB = pd.concat([monthly_ret_XB, monthly_deltas], axis=1).dropna()
df_m_XB.index = df_m_XB.index.strftime("%Y-%m")
df_m_XB

<ipython-input-11-a94e08d6d561>:16: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  .resample("M")     # month-end; you can also use "ME" for explicit MonthEnd
<ipython-input-11-a94e08d6d561>:25: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  .resample("M")


,Ret_m,MM_Δ,Swap_Δ,Prod_Δ,Other_Δ,NonRep_Δ
COT_Date,,,,,,
2006-06,0.098388,0.085394,-0.015535,-0.059429,-0.720183,-3.724444
2006-07,-0.005116,-0.305180,0.123082,0.025504,2.092574,-0.708249
2006-08,-0.290760,0.540467,0.521055,0.513852,0.461198,-1.960889
2006-09,-0.109652,-0.105147,0.112198,0.110988,0.354401,6.659403
2006-10,0.025032,-0.786523,0.136267,-0.070082,0.784841,1.261165
...,...,...,...,...,...,...
2024-12,0.032715,-0.169931,1.748347,0.060756,-1.390064,0.706988
2025-01,0.035725,-0.095527,0.551764,0.055910,-0.024717,5.311146
2025-02,0.045355,-0.094025,0.370914,0.035155,1.573635,0.497210


In [12]:
# wti combined df
inv_wti = inv_wti.set_index("COT_Date")
df_m_wti.index.name = "COT_Date"
df_combined_wti = df_m_wti.join(inv_wti, how="inner")
df_combined_wti

,Ret_m,MM_Δ,Swap_Δ,Prod_Δ,Other_Δ,NonRep_Δ,Inventory
COT_Date,,,,,,,
2009-07,0.062323,0.000000,0.000000,0.000000,0.000000,0.000000,327.2
2009-08,-0.047186,0.393580,-0.509397,-0.275579,-0.093420,-2.761754,317.5
2009-09,0.041587,-0.274340,0.059355,-0.379426,0.364418,8.965208,317.1
2009-10,0.123025,0.182130,0.369218,0.783571,-0.622787,-12.040907,314.4
2009-11,-0.015452,-0.645380,0.100711,-0.189565,0.438153,1.906155,318.8
...,...,...,...,...,...,...,...
2024-12,0.061624,-0.198787,-0.094822,-0.856296,0.741467,1.772403,413.7
2025-01,-0.020875,-0.653224,-0.260872,1.166076,-0.846702,1.938516,418.8
2025-02,-0.061073,0.711218,0.479311,1.266449,-0.601381,1.326063,435.2


In [13]:
# gasoline combined df
inv_xb = inv_xb.set_index("COT_Date")
df_m_XB.index.name = "COT_Date"
df_combined_XB = df_m_XB.join(inv_xb, how="inner")
df_combined_XB

,Ret_m,MM_Δ,Swap_Δ,Prod_Δ,Other_Δ,NonRep_Δ,Inventory
COT_Date,,,,,,,
2006-06,0.098388,0.085394,-0.015535,-0.059429,-0.720183,-3.724444,213.3
2006-07,-0.005116,-0.305180,0.123082,0.025504,2.092574,-0.708249,208.9
2006-08,-0.290760,0.540467,0.521055,0.513852,0.461198,-1.960889,209.0
2006-09,-0.109652,-0.105147,0.112198,0.110988,0.354401,6.659403,214.1
2006-10,0.025032,-0.786523,0.136267,-0.070082,0.784841,1.261165,204.6
...,...,...,...,...,...,...,...
2024-12,0.032715,-0.169931,1.748347,0.060756,-1.390064,0.706988,238.6
2025-01,0.035725,-0.095527,0.551764,0.055910,-0.024717,5.311146,251.1
2025-02,0.045355,-0.094025,0.370914,0.035155,1.573635,0.497210,241.1


In [15]:
# rolling z‑scores over 12 months
net_cols = ["MM_Δ","Swap_Δ","Prod_Δ","Other_Δ","NonRep_Δ","Inventory"]

window = 12

def add_z(df):
    for col in net_cols:
        m = df[col].rolling(window, min_periods=6).mean()
        s = df[col].rolling(window, min_periods=6).std()
        df[col + "_z"] = (df[col] - m) / s
    return df

df_combined_wti = add_z(df_combined_wti)
df_combined_wti

,Ret_m,MM_Δ,Swap_Δ,Prod_Δ,Other_Δ,NonRep_Δ,Inventory,MM_Δ_z,Swap_Δ_z,Prod_Δ_z,Other_Δ_z,NonRep_Δ_z,Inventory_z
COT_Date,,,,,,,,,,,,,
2009-07,0.062323,0.000000,0.000000,0.000000,0.000000,0.000000,327.2,NaN,NaN,NaN,NaN,NaN,NaN
2009-08,-0.047186,0.393580,-0.509397,-0.275579,-0.093420,-2.761754,317.5,NaN,NaN,NaN,NaN,NaN,NaN
2009-09,0.041587,-0.274340,0.059355,-0.379426,0.364418,8.965208,317.1,NaN,NaN,NaN,NaN,NaN,NaN
2009-10,0.123025,0.182130,0.369218,0.783571,-0.622787,-12.040907,314.4,NaN,NaN,NaN,NaN,NaN,NaN
2009-11,-0.015452,-0.645380,0.100711,-0.189565,0.438153,1.906155,318.8,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-12,0.061624,-0.198787,-0.094822,-0.856296,0.741467,1.772403,413.7,-0.823959,-0.467756,0.088563,0.867864,-0.259095,-1.176729
2025-01,-0.020875,-0.653224,-0.260872,1.166076,-0.846702,1.938516,418.8,-1.872985,-0.851003,0.349923,-1.485423,-0.262032,-0.803873
2025-02,-0.061073,0.711218,0.479311,1.266449,-0.601381,1.326063,435.2,1.619082,0.710775,0.359887,-0.978182,-0.262731,0.219521


In [17]:
df_combined_XB = add_z(df_combined_XB)
df_combined_XB

,Ret_m,MM_Δ,Swap_Δ,Prod_Δ,Other_Δ,NonRep_Δ,Inventory,MM_Δ_z,Swap_Δ_z,Prod_Δ_z,Other_Δ_z,NonRep_Δ_z,Inventory_z
COT_Date,,,,,,,,,,,,,
2006-06,0.098388,0.085394,-0.015535,-0.059429,-0.720183,-3.724444,213.3,NaN,NaN,NaN,NaN,NaN,NaN
2006-07,-0.005116,-0.305180,0.123082,0.025504,2.092574,-0.708249,208.9,NaN,NaN,NaN,NaN,NaN,NaN
2006-08,-0.290760,0.540467,0.521055,0.513852,0.461198,-1.960889,209.0,NaN,NaN,NaN,NaN,NaN,NaN
2006-09,-0.109652,-0.105147,0.112198,0.110988,0.354401,6.659403,214.1,NaN,NaN,NaN,NaN,NaN,NaN
2006-10,0.025032,-0.786523,0.136267,-0.070082,0.784841,1.261165,204.6,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-12,0.032715,-0.169931,1.748347,0.060756,-1.390064,0.706988,238.6,-0.593561,2.250355,0.158338,0.329734,0.277929,0.794834
2025-01,0.035725,-0.095527,0.551764,0.055910,-0.024717,5.311146,251.1,-0.457544,0.533880,0.131740,0.482008,0.540870,2.001084
2025-02,0.045355,-0.094025,0.370914,0.035155,1.573635,0.497210,241.1,-0.451023,0.239565,0.034428,0.620810,0.232155,1.043473


In [24]:
# Specifycolumn names
signal_cols  = ["MM_Δ_z", "Swap_Δ_z", "Prod_Δ_z", "Other_Δ_z", "NonRep_Δ_z"]
storage_col  = "Inventory_z"
horizons     = [1, 2, 3]

def run_horizons(df, label):
    """
    For each horizon h, regress Ret_m+h on:
      1) Inventory_z only
      2) signals + Inventory_z
      3) signals + Inventory_z + signal×Inventory interactions
    Prints R² and coefficient tables for each.
    Returns a dict of fitted models.
    """
    models = {}
    for h in horizons:
        # 2) Define the dependent variable: h‑months‑ahead return
        y = df["Ret_m"].shift(-h)

        # 3) Build a working DataFrame with signals, storage, and y
        data = df[signal_cols + [storage_col]].copy()
        data["y"] = y
        data = data.dropna()

        # 4a) Inventory only
        X1 = sm.add_constant(data[[storage_col]])
        m1 = sm.OLS(data["y"], X1).fit(cov_type="HAC", cov_kwds={"maxlags":1})
        print(f"\n--- {label}  t+{h} months: Inventory only (R² = {m1.rsquared:.4f}) ---")
        print(m1.summary().tables[1])

        # 4b) Signals + Inventory
        X2 = sm.add_constant(data[signal_cols + [storage_col]])
        m2 = sm.OLS(data["y"], X2).fit(cov_type="HAC", cov_kwds={"maxlags":1})
        print(f"\n--- {label}  t+{h} months: Signals + Inventory (R² = {m2.rsquared:.4f}) ---")
        print(m2.summary().tables[1])

        # 4c) Add interaction terms: signal_z × Inventory_z
        for col in signal_cols:
            data[f"{col}_x_{storage_col}"] = data[col] * data[storage_col]
        inter_cols = [f"{col}_x_{storage_col}" for col in signal_cols]

        X3 = sm.add_constant(data[signal_cols + [storage_col] + inter_cols])
        m3 = sm.OLS(data["y"], X3).fit(cov_type="HAC", cov_kwds={"maxlags":1})
        print(f"\n--- {label}  t+{h} months: Interactions (R² = {m3.rsquared:.4f}) ---")
        print(m3.summary().tables[1])

        # 5) Store models
        models[h] = {"inv": m1, "sig+stor": m2, "int": m3}

    return models

In [25]:
models_wti = run_horizons(df_combined_wti, "WTI Crude")
models_wti


--- WTI Crude  t+1 months: Inventory only (R² = 0.0002) ---
                  coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------
const           0.0039      0.008      0.519      0.604      -0.011       0.019
Inventory_z    -0.0011      0.008     -0.149      0.882      -0.016       0.014

--- WTI Crude  t+1 months: Signals + Inventory (R² = 0.0147) ---
                  coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------
const           0.0030      0.007      0.399      0.690      -0.012       0.017
MM_Δ_z         -0.0002      0.006     -0.033      0.974      -0.012       0.011
Swap_Δ_z       -0.0094      0.008     -1.167      0.243      -0.025       0.006
Prod_Δ_z       -0.0029      0.008     -0.354      0.723      -0.019       0.013
Other_Δ_z      -0.0050      0.008     -0.666      0.505      -0.020      

{1: {'inv': <statsmodels.regression.linear_model.RegressionResultsWrapper at 0x7bf66c88e650>,
  'sig+stor': <statsmodels.regression.linear_model.RegressionResultsWrapper at 0x7bf66c886510>,
  'int': <statsmodels.regression.linear_model.RegressionResultsWrapper at 0x7bf66c8a3650>},
 2: {'inv': <statsmodels.regression.linear_model.RegressionResultsWrapper at 0x7bf66c8cc310>,
  'sig+stor': <statsmodels.regression.linear_model.RegressionResultsWrapper at 0x7bf66c89f750>,
  'int': <statsmodels.regression.linear_model.RegressionResultsWrapper at 0x7bf66f2cce10>},
 3: {'inv': <statsmodels.regression.linear_model.RegressionResultsWrapper at 0x7bf66cc6ee90>,
  'sig+stor': <statsmodels.regression.linear_model.RegressionResultsWrapper at 0x7bf66c8734d0>,
  'int': <statsmodels.regression.linear_model.RegressionResultsWrapper at 0x7bf66c8f9650>}}

In [30]:
models_xb = run_horizons(df_combined_XB, "GBOB Gasoline")
models_xb


--- GBOB Gasoline  t+1 months: Inventory only (R² = 0.1125) ---
                  coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------
const           0.0067      0.007      0.934      0.351      -0.007       0.021
Inventory_z     0.0370      0.007      5.223      0.000       0.023       0.051

--- GBOB Gasoline  t+1 months: Signals + Inventory (R² = 0.1419) ---
                  coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------
const           0.0072      0.007      1.021      0.307      -0.007       0.021
MM_Δ_z         -0.0121      0.010     -1.213      0.225      -0.032       0.007
Swap_Δ_z       -0.0010      0.007     -0.144      0.885      -0.014       0.012
Prod_Δ_z        0.0255      0.009      2.788      0.005       0.008       0.043
Other_Δ_z      -0.0023      0.006     -0.420      0.675      -0.0

{1: {'inv': <statsmodels.regression.linear_model.RegressionResultsWrapper at 0x7bf66c8ed250>,
  'sig+stor': <statsmodels.regression.linear_model.RegressionResultsWrapper at 0x7bf66b66d910>,
  'int': <statsmodels.regression.linear_model.RegressionResultsWrapper at 0x7bf66ac575d0>},
 2: {'inv': <statsmodels.regression.linear_model.RegressionResultsWrapper at 0x7bf66afa6290>,
  'sig+stor': <statsmodels.regression.linear_model.RegressionResultsWrapper at 0x7bf66b633d10>,
  'int': <statsmodels.regression.linear_model.RegressionResultsWrapper at 0x7bf66af4add0>},
 3: {'inv': <statsmodels.regression.linear_model.RegressionResultsWrapper at 0x7bf66afd1310>,
  'sig+stor': <statsmodels.regression.linear_model.RegressionResultsWrapper at 0x7bf66ac4e5d0>,
  'int': <statsmodels.regression.linear_model.RegressionResultsWrapper at 0x7bf66ac52610>}}

In [26]:
from sklearn.linear_model import LassoCV
from sklearn.preprocessing import StandardScaler

def feature_select_and_fit(df, label, h, p_thresh=0.05):
    """
    For horizon h, build the full interaction design matrix,
    then apply:
      a) backward‐elimination by p‐value
      b) LassoCV selection
    Print R² and coeff table for each.
    Return both fitted models.
    """
    # dependent: h‐ahead return
    y = df["Ret_m"].shift(-h)

    # build base data
    data = df[signal_cols + [storage_col]].copy()
    data["y"] = y
    data = data.dropna()

    # build interaction terms
    for col in signal_cols:
        data[f"{col}_x_{storage_col}"] = data[col] * data[storage_col]
    inter_cols = [f"{c}_x_{storage_col}" for c in signal_cols]

    # full X matrix
    X_full = sm.add_constant(data[signal_cols + [storage_col] + inter_cols])
    y_full = data["y"]

    # --- a) Backward‐elimination by p‐value ----------------------------
    X_be = X_full.copy()
    while True:
        m = sm.OLS(y_full, X_be).fit(cov_type="HAC", cov_kwds={"maxlags":1})
        pvals = m.pvalues.drop("const")
        worst_p = pvals.max()
        if worst_p > p_thresh:
            X_be = X_be.drop(columns=[pvals.idxmax()])
        else:
            break
    be_model = sm.OLS(y_full, X_be).fit(cov_type="HAC", cov_kwds={"maxlags":1})

    # --- b) LassoCV selection ------------------------------------------
    # scale features (exclude constant)
    X_noc = X_full.drop(columns="const")
    scaler = StandardScaler()
    Xs = scaler.fit_transform(X_noc)
    lasso = LassoCV(cv=5).fit(Xs, y_full)
    coef = pd.Series(lasso.coef_, index=X_noc.columns)
    selected = coef[coef != 0].index.tolist()
    X_lasso = sm.add_constant(X_noc[selected])
    lasso_model = sm.OLS(y_full, X_lasso).fit(cov_type="HAC", cov_kwds={"maxlags":1})

    # --- Print results ------------------------------------------------
    print(f"\n=== {label} t+{h}‑month Feature Selection ===")

    print(f"\n-- Backward Elimination (p>{p_thresh} removed) R²={be_model.rsquared:.4f} --")
    print("Features:", X_be.columns.tolist())
    print(be_model.summary().tables[1])

    print(f"\n-- LassoCV Selection R²={lasso_model.rsquared:.4f} --")
    print("Features:", ["const"] + selected)
    print(lasso_model.summary().tables[1])

    return {"backward": be_model, "lasso": lasso_model}

In [27]:
models_wti_fs = {h: feature_select_and_fit(df_combined_wti, "WTI Crude",    h) for h in horizons}
models_wti_fs


=== WTI Crude t+1‑month Feature Selection ===

-- Backward Elimination (p>0.05 removed) R²=0.0469 --
Features: ['const', 'Other_Δ_z_x_Inventory_z']
                              coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------------
const                       0.0028      0.008      0.373      0.709      -0.012       0.018
Other_Δ_z_x_Inventory_z    -0.0164      0.005     -3.194      0.001      -0.027      -0.006

-- LassoCV Selection R²=0.0577 --
Features: ['const', 'Swap_Δ_z', 'Other_Δ_z_x_Inventory_z']
                              coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------------
const                       0.0022      0.007      0.295      0.768      -0.012       0.017
Swap_Δ_z                   -0.0102      0.008     -1.313      0.189      -0.025       0.005
Other_Δ_z_x_Inventory

{1: {'backward': <statsmodels.regression.linear_model.RegressionResultsWrapper at 0x7bf66c911910>,
  'lasso': <statsmodels.regression.linear_model.RegressionResultsWrapper at 0x7bf66c8a6b90>},
 2: {'backward': <statsmodels.regression.linear_model.RegressionResultsWrapper at 0x7bf66b5bea90>,
  'lasso': <statsmodels.regression.linear_model.RegressionResultsWrapper at 0x7bf66bc99050>},
 3: {'backward': <statsmodels.regression.linear_model.RegressionResultsWrapper at 0x7bf66b631c50>,
  'lasso': <statsmodels.regression.linear_model.RegressionResultsWrapper at 0x7bf66bc30c10>}}

In [31]:
models_xb_fs = {h: feature_select_and_fit(df_combined_XB, "GBOB Gasoline",    h) for h in horizons}
models_xb_fs


=== GBOB Gasoline t+1‑month Feature Selection ===

-- Backward Elimination (p>0.05 removed) R²=0.1535 --
Features: ['const', 'Prod_Δ_z', 'Inventory_z', 'MM_Δ_z_x_Inventory_z']
                           coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------------
const                    0.0070      0.007      1.013      0.311      -0.007       0.021
Prod_Δ_z                 0.0182      0.006      2.982      0.003       0.006       0.030
Inventory_z              0.0367      0.007      5.064      0.000       0.023       0.051
MM_Δ_z_x_Inventory_z    -0.0173      0.008     -2.095      0.036      -0.033      -0.001

-- LassoCV Selection R²=0.1743 --
Features: ['const', 'MM_Δ_z', 'Prod_Δ_z', 'Inventory_z', 'MM_Δ_z_x_Inventory_z', 'Prod_Δ_z_x_Inventory_z', 'Other_Δ_z_x_Inventory_z', 'NonRep_Δ_z_x_Inventory_z']
                               coef    std err          z      P>|z|      [0.025      0.975]

{1: {'backward': <statsmodels.regression.linear_model.RegressionResultsWrapper at 0x7bf66ac56250>,
  'lasso': <statsmodels.regression.linear_model.RegressionResultsWrapper at 0x7bf66c91ce50>},
 2: {'backward': <statsmodels.regression.linear_model.RegressionResultsWrapper at 0x7bf66ac45350>,
  'lasso': <statsmodels.regression.linear_model.RegressionResultsWrapper at 0x7bf66ac7a850>},
 3: {'backward': <statsmodels.regression.linear_model.RegressionResultsWrapper at 0x7bf66ac4e290>,
  'lasso': <statsmodels.regression.linear_model.RegressionResultsWrapper at 0x7bf66ac4fe50>}}

In [28]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score


def run_random_forest(df, label):
    """
    For each horizon h, fit a RandomForestRegressor to predict Ret_m+h from:
      • signal_cols
      • storage_col
      • all interactions signal×storage
    Prints in‑sample R² and feature importances.
    Returns a dict of fitted RF models.
    """
    rf_models = {}
    for h in horizons:
        # Build dependent variable y = Ret_m shifted by –h
        y = df["Ret_m"].shift(-h)

        # Build feature DataFrame with signals, storage, and interactions
        X = df[signal_cols + [storage_col]].copy()
        for col in signal_cols:
            X[f"{col}_x_{storage_col}"] = X[col] * X[storage_col]
        X["y"] = y
        X = X.dropna()

        # Separate features and target
        y_train = X.pop("y")
        X_train = X

        # Fit Random Forest
        rf = RandomForestRegressor(
            n_estimators=200,
            min_samples_leaf=5,
            random_state=0,
            n_jobs=-1
        )
        rf.fit(X_train, y_train)

        # In‑sample R²
        y_pred = rf.predict(X_train)
        r2 = r2_score(y_train, y_pred)
        print(f"\n--- {label} t+{h} months Random Forest — R² = {r2:.4f} ---")

        # Feature importances
        imps = pd.Series(rf.feature_importances_, index=X_train.columns)
        imps = imps.sort_values(ascending=False)
        print(imps)

        # Store the model
        rf_models[h] = rf

    return rf_models

In [29]:
models_wti_rf = run_random_forest(df_combined_wti, "WTI Crude")
models_wti_rf


--- WTI Crude t+1 months Random Forest — R² = 0.5166 ---
Other_Δ_z_x_Inventory_z     0.215297
Swap_Δ_z                    0.101875
Prod_Δ_z                    0.087671
NonRep_Δ_z_x_Inventory_z    0.086360
Swap_Δ_z_x_Inventory_z      0.085893
NonRep_Δ_z                  0.080904
MM_Δ_z_x_Inventory_z        0.076902
MM_Δ_z                      0.074376
Prod_Δ_z_x_Inventory_z      0.070159
Inventory_z                 0.066438
Other_Δ_z                   0.054126
dtype: float64

--- WTI Crude t+2 months Random Forest — R² = 0.5025 ---
Swap_Δ_z_x_Inventory_z      0.174551
NonRep_Δ_z                  0.145911
Other_Δ_z_x_Inventory_z     0.135240
NonRep_Δ_z_x_Inventory_z    0.116181
Prod_Δ_z                    0.075735
Swap_Δ_z                    0.075071
Prod_Δ_z_x_Inventory_z      0.073427
MM_Δ_z                      0.060914
Other_Δ_z                   0.058872
MM_Δ_z_x_Inventory_z        0.045151
Inventory_z                 0.038947
dtype: float64

--- WTI Crude t+3 months Random Forest 

{1: RandomForestRegressor(min_samples_leaf=5, n_estimators=200, n_jobs=-1,
                       random_state=0),
 2: RandomForestRegressor(min_samples_leaf=5, n_estimators=200, n_jobs=-1,
                       random_state=0),
 3: RandomForestRegressor(min_samples_leaf=5, n_estimators=200, n_jobs=-1,
                       random_state=0)}

In [32]:
models_xb_rf = run_random_forest(df_combined_XB, "GBOB Gasoline")
models_xb_rf


--- GBOB Gasoline t+1 months Random Forest — R² = 0.5618 ---
Inventory_z                 0.312087
Prod_Δ_z                    0.103692
NonRep_Δ_z                  0.079382
MM_Δ_z_x_Inventory_z        0.077818
NonRep_Δ_z_x_Inventory_z    0.070164
Prod_Δ_z_x_Inventory_z      0.068516
Other_Δ_z_x_Inventory_z     0.064146
Swap_Δ_z_x_Inventory_z      0.062511
Other_Δ_z                   0.058552
Swap_Δ_z                    0.056317
MM_Δ_z                      0.046815
dtype: float64

--- GBOB Gasoline t+2 months Random Forest — R² = 0.5336 ---
NonRep_Δ_z_x_Inventory_z    0.132866
Swap_Δ_z                    0.131815
Inventory_z                 0.128174
Other_Δ_z                   0.093101
Prod_Δ_z                    0.091564
NonRep_Δ_z                  0.087023
Prod_Δ_z_x_Inventory_z      0.080788
MM_Δ_z                      0.078610
Swap_Δ_z_x_Inventory_z      0.062027
MM_Δ_z_x_Inventory_z        0.057883
Other_Δ_z_x_Inventory_z     0.056149
dtype: float64

--- GBOB Gasoline t+3 months Ra

{1: RandomForestRegressor(min_samples_leaf=5, n_estimators=200, n_jobs=-1,
                       random_state=0),
 2: RandomForestRegressor(min_samples_leaf=5, n_estimators=200, n_jobs=-1,
                       random_state=0),
 3: RandomForestRegressor(min_samples_leaf=5, n_estimators=200, n_jobs=-1,
                       random_state=0)}